In [5]:
import pandas as pd, sqlite3

# Ex02. Join

## 1. Подключение к базе данных

In [6]:
conn = sqlite3.connect("../data/checking-logs.sqlite")
cursor = conn.cursor()

## 2. Новая таблица datamart

In [7]:
cursor.execute('''
               CREATE TABLE datamart AS
               SELECT c.uid, c.labname, MIN(c.timestamp) first_commit_ts, MIN(p.datetime) first_view_ts
               FROM checker c
               LEFT JOIN pageviews p ON c.uid = p.uid 
               WHERE c.uid LIKE 'user_%'
                    and c.status = 'ready'
                    and c.numTrials = 1
                    and c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
               GROUP BY c.uid, c.labname
               ''')
conn.commit()

## 3. Новые таблицы test и control

In [8]:
datamart = pd.DataFrame(cursor.execute('SELECT * FROM datamart'), columns=['uid', 'labname', 'first_commit_ts', 'first_view_ts'])
datamart[['first_commit_ts', 'first_view_ts']] = datamart[['first_commit_ts', 'first_view_ts']].astype('datetime64[ns]')

In [9]:
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


In [10]:
test = datamart.dropna(subset='first_view_ts')
control = datamart[datamart.first_view_ts.isna()]

In [11]:
mean_value = test['first_view_ts'].mean()
control = control.fillna({'first_view_ts': mean_value})

**Добавление таблиц в базу данных**

In [12]:
test.to_sql('test', con=conn, if_exists='replace')


59

In [13]:
control.to_sql('control', con=conn, if_exists='replace')

81

## 4. Закрытие соединения

In [14]:
conn.commit()
conn.close()